Description:
 - This notebook demonstrates a Foundry agent that connects to a sample MCP server.
 
 Setup:
 - For notebook prerequisites, please see: https://code.visualstudio.com/docs/languages/polyglot
 - In the "//Create a project client" cell, replace "YOUR-FOUNDARY-PROJECT-ENDPOINT" with your Foundry project endpoint URL.
 - Set the execution environment for this notebook to .NET Interactive.
 
 Run:
 - To run, execute each cell in order.

 Repo Structure
 --------------
 This notebook is part of a repo that has two folders:

  - Client: This folder. Contains this notebook.
  
  - Server: GitHub repo that implements an MCP server as an Azure Function. The initial version gets the weather using MCP. I have modified this repo to add pizza ordering functionality. See Server/Readme.txt for details on the server component.

In [ ]:
// Add NuGet package references.
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.5"
#r "nuget: Azure.Identity, 1.12.1"

In [ ]:
// Add using statements.
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Azure;
using System;
using System.Collections.Generic;
using System.Threading;

In [ ]:
//Create a project client
var PROJECT_ENDPOINT        = "YOUR-FOUNDARY-PROJECT-ENDPOINT";      //TODO!: Create a Foundry project and get the endpoint URL.
var MODEL_DEPLOYMENT_NAME   = "gpt-4.1";
var MCP_SERVER_URL          = "https://learn.microsoft.com/api/mcp"; //URL to sample MCP server.
var MCP_SERVER_LABEL        = "GitHub_mcp_server_label";             //MCP server friendly name.

var projectEndpoint         = PROJECT_ENDPOINT;
var modelDeploymentName     = MODEL_DEPLOYMENT_NAME;
var mcpServerUrl            = MCP_SERVER_URL;
var mcpServerLabel          = MCP_SERVER_LABEL;

PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

In [ ]:
// Create MCP tool definition
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

// Configure allowed tools (optional)
string searchApiCode = "search_azure_rest_api_code";
mcpTool.AllowedTools.Add(searchApiCode);

In [ ]:
//Use the MCPToolDefinition during the agent initialization.

//create a unique identifier for the agent name
var agentname = "my-mcp-agent-" + DateTime.UtcNow.ToString("yyyy-MM-dd_HH:mm");
 
PersistentAgent agent = agentClient.Administration.CreateAgent(
   model: modelDeploymentName,
   name: agentname,
   //instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   instructions: "You are a helpful agent.",
   tools: [mcpTool]);

In [ ]:
//Create a thread and add a message.

PersistentAgentThread thread = agentClient.Threads.CreateThread();

// Create message to thread
PersistentThreadMessage message = agentClient.Messages.CreateMessage(
    thread.Id,
    MessageRole.User,
    "Please summarize the Azure REST API specifications Readme");

MCPToolResource mcpToolResource = new(mcpServerLabel);
mcpToolResource.UpdateHeader("SuperSecret", "123456");
ToolResources toolResources = mcpToolResource.ToToolResources();

// Run the agent with MCP tool resources
ThreadRun run = agentClient.Runs.CreateRun(thread, agent, toolResources);

// Handle run execution and tool approvals
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    Thread.Sleep(TimeSpan.FromMilliseconds(1000));
    run = agentClient.Runs.GetRun(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}, Arguments: {mcpToolCall.Arguments}");
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                {
                    Headers = { ["SuperSecret"] = "123456" }
                });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = agentClient.Runs.SubmitToolOutputsToRun(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

In [ ]:
//Print the messages to the console.
//Note: C# notebooks do not handle streaming responses, the response occurs all at once after the call is complete.

Pageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessages(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

In [ ]:
//Optional: Delete the agent.
//When running the notebook, I suggest deleting the agent each time (each run will create a new agent.)
//Use the Azure AI Foundry Extension to view your projects, models, agents, threads, etc.

agentClient.Threads.DeleteThread(threadId: thread.Id);
agentClient.Administration.DeleteAgent(agentId: agent.Id);